# showus

In [ ]:
#default_exp showus

In [ ]:
! pip install /kaggle/input/nlp-packages/datasets/datasets/fsspec-2021.4.0-py3-none-any.whl
! pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
! pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
! pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Processing /kaggle/input/nlp-packages/datasets/datasets/fsspec-2021.4.0-py3-none-any.whl
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.8.7
    Uninstalling fsspec-0.8.7:
      Successfully uninstalled fsspec-0.8.7
Looking in links: file:///kaggle/input/coleridge-packages/packages/datasets
Processing /kaggle/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/tqdm-4.49.0-py2.py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
Processing /kaggle/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/tokenizers-0.10.

In [ ]:
#export
import os, shutil
from tqdm import tqdm
from pathlib import Path
import itertools
from functools import partial
import re
import json
import random
import numpy as np
import pandas as pd
import torch
import transformers, seqeval
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, ClassLabel, load_metric

In [ ]:
# !cp /kaggle/input/coleridge-packages/my_seqeval.py ./

# Utilities

In [ ]:
#export
Path.ls = lambda pth: list(pth.iterdir())

# Data I/O

In [ ]:
#export
def load_train_meta(pth, group_id=True):
    df = pd.read_csv(pth)
    if group_id:
        df = df.groupby('Id').agg({'pub_title': 'first', 'dataset_title': '|'.join, 
                                   'dataset_label': '|'.join, 'cleaned_label': '|'.join}).reset_index()
    return df

In [ ]:
pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
df = load_train_meta(pth, group_id=True)
df_nogroup = load_train_meta(pth, group_id=False)
print(len(df), len(df_nogroup))
dup_ids = df_nogroup[df_nogroup.Id.duplicated()].Id.unique()
print(df[df.Id.isin(dup_ids)].dataset_label.values[-10:])

14316 19661
['Baltimore Longitudinal Study of Aging (BLSA)|Baltimore Longitudinal Study of Aging'
 'Beginning Postsecondary Students Longitudinal Study|Education Longitudinal Study|Beginning Postsecondary Students'
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 'Baltimore Longitudinal Study of Aging (BLSA)|Baltimore Longitudinal Study of Aging'
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 'Beginning Postsecondary Student|Beginning Postsecondary Students']


In [ ]:
#export
def load_papers(dir_json, paper_ids):
    '''
    Load papers into a dictionary.
    
    `papers`: 
        {''}
    '''
    
    papers = {}
    for paper_id in paper_ids:
        with open(f'{dir_json}/{paper_id}.json', 'r') as f:
            paper = json.load(f)
            papers[paper_id] = paper
    return papers

In [ ]:
df = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id.unique()[:10])
print(type(papers))
print(
    papers[ random.choice(list(papers.keys())) ][0]
)

<class 'dict'>
{'section_title': 'Abstract', 'text': "In this article, the authors report the results of two studies examining the participation rates of Latino students in postsecondary technical education (CTE) programs in community colleges and two-year proprietary institutions in the United States in 1994 and 2000. It is believed that the quality of the future U.S. Labor market will depend, to a great extent, on this group's education and job skills. Although Latinos are the fastest growing minority group in the United States, they are also the poorest and most undereducated when compared to other minority groups. Results of both studies show that few Latino students enroll in and graduate from postsecondary CTE programs. Of those students that do enroll in and complete CTE programs at the postsecondary level, very few complete programs that are considered high-skill, high-wage."}


In [ ]:
#export
def load_sample_text(jpth):
    sections = json.loads(jpth.read_text())
    text = '\n'.join(section['text'] for section in sections)
    return text

In [ ]:
jpths_trn = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train/').ls()
print(load_sample_text(jpths_trn[0])[:1_000])

The International Standard Classification of Education, known by its acronym ISCED, was developed by the United Nations Educational, Scientific, and Cultural Organization during the late 1960s and 1970s. ISCED was implemented in 1976 and is the recognized international standard for reporting and interpreting education program data. Creating a U.S. crosswalk to this system has been a goal of the National Center for Education Statistics and the Office of Research since the late 197,,s, when the National Institute of Education (the predecessor agency to the Office of Educational Research and Improvement) began exploring the idea. The design and implementation of a workable crosswalk, however, awaited the advent of changes to the Classification of Instructional Programs (CIP) system. The 1990 revision of the CIP system laid the foundation for a workable international crosswalk. Adoption of the National Education Goals set global consciousness and international educational comparisons firml

# Data processing

In [ ]:
#export
def clean_training_text(txt, lower=False, total_clean=False):
    """
    similar to the default clean_text function but without lowercasing.
    """
    txt = str(txt).lower() if lower else str(txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt).strip()
    if total_clean:
        txt = re.sub(' +', ' ', txt)
    return txt

In [ ]:
print(clean_training_text('@kaggle This competition awards $90,000!!!!.'))
print(clean_training_text('HoPKLd + 7 ! 11,002', total_clean=True, lower=True))

kaggle This competition awards 90 000
hopkld 7 11 002


In [ ]:
#export
def shorten_sentences(sentences, max_length=64, overlap=20):
    '''
    Args:
        sentences (list): List of sentences.
        max_length (int): Maximum number of words allowed for each sentence.
        overlap (int): If a sentence exceeds `max_length`, we split it to multiple sentences with 
            this amount of overlapping.
    '''
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > max_length:
            for p in range(0, len(words), max_length - overlap):
                short_sentences.append(' '.join(words[p:p+max_length]))
        else:
            short_sentences.append(sentence)
    return short_sentences

In [ ]:
jpths_trn = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train/').ls()
sentences = load_sample_text(jpths_trn[0]).split('.')[:2]
short_sentences = shorten_sentences(sentences, max_length=10, overlap=2)
print('Before:', sentences)
print()
print('After:', short_sentences)

Before: ['The International Standard Classification of Education, known by its acronym ISCED, was developed by the United Nations Educational, Scientific, and Cultural Organization during the late 1960s and 1970s', ' ISCED was implemented in 1976 and is the recognized international standard for reporting and interpreting education program data']

After: ['The International Standard Classification of Education, known by its acronym', 'its acronym ISCED, was developed by the United Nations Educational,', 'Nations Educational, Scientific, and Cultural Organization during the late 1960s', 'late 1960s and 1970s', 'ISCED was implemented in 1976 and is the recognized international', 'recognized international standard for reporting and interpreting education program data', 'program data']


In [ ]:
#export
def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

In [ ]:
big_list = ['If', 'the', 'thing', 'above', 'is', 'below', 'that', 'thing', 'which', 'is',
            'not', 'as', 'high', 'up', 'on', 'the', 'thing', 'above', 'when', 'it', 'is', 
            'underneath', 'them.']
small_list = ['the', 'thing', 'above']

find_sublist(big_list, small_list)

[1, 15]

# Named Entity Recognition

In [ ]:
#export
def get_ner_classlabel():
    '''
    Labels for named entity recognition.
        'O': Token not part of a phrase that mentions a dataset.
        'I': Intermediate token of a phrase mentioning a dataset.
        'B': First token of a phrase mentioning a dataset.
    '''
    return ClassLabel(names=['O', 'I', 'B'])

In [ ]:
classlabel = get_ner_classlabel()
print(classlabel)
print(classlabel.str2int(['I', 'O', 'B']), classlabel.str2int('I'))
print(classlabel.int2str(2), classlabel.int2str([2, 1, 0]))

ClassLabel(num_classes=3, names=['O', 'I', 'B'], names_file=None, id=None)
[1, 0, 2] 1
B ['B', 'I', 'O']


In [ ]:
#export
def tag_sentence(sentence, labels, classlabel=None): 
    '''
    requirement: both sentence and labels are already cleaned
    '''
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = [classlabel.str2int('O')] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = classlabel.str2int('B')
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = classlabel.str2int('I')

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = [classlabel.str2int('O')] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [ ]:
sentence = ("The International Standard Classification of Education, known by its acronym ISCED, "
            "was developed by the United Nations Educational, "
            "Scientific, and Cultural Organization during the late 1960s and 1970s")
labels = ['The International', 'Cultural Organization', 'United Nations Educational']

sentence = clean_training_text(sentence)
labels = [clean_training_text(label) for label in labels]
classlabel = get_ner_classlabel()
found_any, token_tags = tag_sentence(sentence, labels, classlabel=classlabel)

print('A label is found in the sentence:', found_any)
print('(token, tag) pairs:')
print(token_tags)

A label is found in the sentence: True
(token, tag) pairs:
[('The', 2), ('International', 1), ('Standard', 0), ('Classification', 0), ('of', 0), ('Education', 0), ('known', 0), ('by', 0), ('its', 0), ('acronym', 0), ('ISCED', 0), ('was', 0), ('developed', 0), ('by', 0), ('the', 0), ('United', 2), ('Nations', 1), ('Educational', 1), ('Scientific', 0), ('and', 0), ('Cultural', 2), ('Organization', 1), ('during', 0), ('the', 0), ('late', 0), ('1960s', 0), ('and', 0), ('1970s', 0)]


In [ ]:
#export
def get_ner_data(papers, df=None, classlabel=None, shuffle=True):
    '''
    Args:
        papers (dict): Like that returned by `load_papers`.
        df (pd.DataFrame): Competition's train.csv or a subset of it.
    '''
    cnt_pos, cnt_neg = 0, 0 
    ner_data = []

    tqdm._instances.clear()
    pbar = tqdm(total=len(df))
    for i, id, dataset_label in df[['Id', 'dataset_label']].itertuples():
        paper = papers[id]

        labels = dataset_label.split('|')
        labels = [clean_training_text(label) for label in labels]

        sentences = set([clean_training_text(sentence) for section in paper 
                     for sentence in section['text'].split('.')])
        sentences = shorten_sentences(sentences) 
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars

        # positive sample
        for sentence in sentences:
            is_positive, tags = tag_sentence(sentence, labels, classlabel=classlabel)
            if is_positive:
                cnt_pos += 1
                ner_data.append(tags)
            elif any(word in sentence.lower() for word in ['data', 'study']): 
                ner_data.append(tags)
                cnt_neg += 1

        pbar.update(1)
        pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")
#         print(f"\rProcessing paper {i:05d} / {len(df)}. Training data size: {cnt_pos} positives + {cnt_neg} negatives", 
#               flush=True, end='')

    if shuffle:
        random.shuffle(ner_data)
    return cnt_pos, cnt_neg, ner_data

In [ ]:
%%time
df = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv').iloc[:100]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id)
classlabel = get_ner_classlabel()
cnt_pos, cnt_neg, ner_data = get_ner_data(papers, df, classlabel=classlabel, shuffle=False)
print(f'Postive count: {cnt_pos}.   Negative count: {cnt_neg}')
print(ner_data[250])

Training data size: 168 positives + 2574 negatives: 100%|██████████| 100/100 [00:00<00:00, 113.29it/s]

Postive count: 168.   Negative count: 2574
[('These', 0), ('works', 0), ('include', 0), ('Coleman', 0), ('and', 0), ('Hoffer', 0), ('s', 0), ('findings', 0), ('regarding', 0), ('the', 0), ('social', 0), ('capital', 0), ('and', 0), ('functional', 0), ('community', 0), ('of', 0), ('Catholic', 0), ('schools', 0), ('Chubb', 0), ('and', 0), ('Moe', 0), ('s', 0), ('conclusions', 0), ('regarding', 0), ('the', 0), ('climate', 0), ('of', 0), ('collegiality', 0), ('and', 0), ('empowerment', 0), ('in', 0), ('Catholic', 0), ('schools', 0), ('the', 0), ('Rand', 0), ('study', 0), ('s', 0), ('focus', 0), ('on', 0), ('clarity', 0), ('of', 0), ('mission', 0), ('and', 0), ('local', 0), ('ownership', 0), ('by', 0), ('educators', 0), ('students', 0), ('and', 0), ('parents', 0), ('and', 0), ('Bryk', 0), ('Lee', 0), ('and', 0), ('Holland', 0), ('s', 0), ('emphasis', 0), ('on', 0), ('Catholic', 0), ('schools', 0), ('as', 0), ('voluntary', 0), ('communities', 0)]
CPU times: user 920 ms, sys: 48.1 ms, total: 9

In [ ]:
#export
def write_ner_json(ner_data, pth=Path('train_ner.json')):
    with open(pth, 'w') as f:
        for row in ner_data:
            words, nes = list(zip(*row))
            row_json = {'tokens' : words, 'ner_tags' : nes}
            json.dump(row_json, f)
            f.write('\n')    

In [ ]:
ner_data = [
    [('There', 0), ('is', 0), ('no', 0), ('dataset', 0), ('here', 0)], 
    [('Load', 0), ('the', 0), ('UN', 2), ('Trade', 1), ('Development', 1), ('into', 0), ('view', 0)]
]
write_ner_json(ner_data, pth=Path('/kaggle/tmp_ner.json'))
! cat /kaggle/tmp_ner.json

{"tokens": ["There", "is", "no", "dataset", "here"], "ner_tags": [0, 0, 0, 0, 0]}
{"tokens": ["Load", "the", "UN", "Trade", "Development", "into", "view"], "ner_tags": [0, 0, 2, 1, 1, 0, 0]}


In [ ]:
#export
def load_ner_datasets(train=None, valid=None):
    datasets = load_dataset('json', data_files={'train': train, 'valid': valid})
    classlabel = get_ner_classlabel()

    for split, dataset in datasets.items():
        dataset.features['ner_tags'].feature = classlabel
    return datasets

In [ ]:
datasets = load_ner_datasets(train='/kaggle/tmp_ner.json', valid='/kaggle/tmp_ner.json')
print(datasets['valid'].features)
print()
print(datasets['train'][1])

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4631cd9faaa49ff6/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=3, names=['O', 'I', 'B'], names_file=None, id=None), length=-1, id=None)}

{'tokens': ['Load', 'the', 'UN', 'Trade', 'Development', 'into', 'view'], 'ner_tags': [0, 0, 2, 1, 1, 0, 0]}


In [ ]:
#export
def create_tokenizer(model_checkpoint='bert-base-cased'):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
    return tokenizer

In [ ]:
tokenizer = create_tokenizer(model_checkpoint='bert-base-cased')
print(
    tokenizer("A smattering of people descended from the midday boat on Monday."))
print()
print(
    tokenizer("Giglio boasts several pristine bays with crystal clear water".split(), is_split_into_words=True)
)


{'input_ids': [101, 138, 188, 21943, 9930, 1104, 1234, 9026, 1121, 1103, 2286, 6194, 3499, 1113, 6356, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

{'input_ids': [101, 144, 6512, 9436, 24372, 1317, 185, 12937, 2042, 15520, 1114, 8626, 2330, 1447, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
#export
def tokenize_and_align_labels(examples, tokenizer=None, label_all_tokens=True):
    '''
    Adds a new field called 'labels' that are the NER tags to the tokenized input.
    
    Args:
        tokenizer (transformers.AutoTokenizer): Tokenizer.
        examples (datasets.arrow_dataset.Dataset): Dataset.
        label_all_tokens (bool): If True, all sub-tokens are given the same tag as the 
            first sub-token, otherwise all but the first sub-token are given the tag
            -100.
    '''
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
datasets = load_ner_datasets(train='/kaggle/tmp_ner.json', valid='/kaggle/tmp_ner.json')
tokenizer = create_tokenizer(model_checkpoint='bert-base-cased')

print(tokenize_and_align_labels(datasets['train'][:], tokenizer, label_all_tokens=True), end='\n\n')

tokenized_datasets = datasets.map(
    partial(tokenize_and_align_labels, tokenizer=tokenizer, label_all_tokens=True), batched=True)

print(tokenized_datasets['train'][:], end='\n\n')
print(tokenized_datasets['valid'][:])
print(tokenized_datasets['train'].features)

{'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]]}




{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]], 'ner_tags': [[0, 0, 0, 0, 0], [0, 0, 2, 1, 1, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'tokens': [['There', 'is', 'no', 'dataset', 'here'], ['Load', 'the', 'UN', 'Trade', 'Development', 'into', 'view']]}

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]], 'ner_tags': [[0, 0, 0, 0, 0], [0, 0, 2, 1, 1, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'tokens': [['There', 'is', 'no', 'dataset', 'he

In [ ]:
classlabel = get_ner_classlabel()
model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=classlabel.num_classes)

Training data size: 168 positives + 2574 negatives: 100%|██████████| 100/100 [00:19<00:00, 113.29it/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
args = TrainingArguments(output_dir='test_ner', evaluation_strategy='epoch', learning_rate=2e-5, 
                         per_device_train_batch_size=8, per_device_eval_batch_size=8, num_train_epochs=3,
                         weight_decay=0.01, report_to='none')

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric('seqeval')

In [ ]:
predictions = [['O', 'O', 'B', 'I', 'I', 'O']]
references = [['B', 'I', 'B', 'O', 'O', 'O']]
metric.compute(predictions=predictions, references=references)

{'_': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.3333333333333333}

In [ ]:
#export
def compute_metrics(p, metric=None, label_list=None):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
! rm -r test_ner

rm: cannot remove 'test_ner': No such file or directory


In [ ]:
classlabel = get_ner_classlabel()
train_meta = load_train_meta('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv').iloc[:50]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train', train_meta.Id)

valid_cutoff = int(.10 * len(train_meta))
valid_meta = train_meta.iloc[:valid_cutoff].reset_index(drop=True)
train_meta = train_meta.iloc[valid_cutoff:].reset_index(drop=True)

train_cnt_pos, train_cnt_neg, train_ner_data = get_ner_data(papers, df=train_meta, classlabel=classlabel)
valid_cnt_pos, valid_cnt_neg, valid_ner_data = get_ner_data(papers, df=valid_meta, classlabel=classlabel)
print(f'Train.  Positive count: {train_cnt_pos}.  Negative count: {train_cnt_neg}.')
print(f'Valid.  Positive count: {valid_cnt_pos}.  Negative count: {valid_cnt_neg}.')

write_ner_json(train_ner_data, pth='train_ner.json')
write_ner_json(valid_ner_data, pth='valid_ner.json')

datasets = load_ner_datasets(train='train_ner.json', valid='valid_ner.json')

model_checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = datasets.map(
    partial(tokenize_and_align_labels, tokenizer=tokenizer, label_all_tokens=True), batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=classlabel.num_classes)

metric = load_metric('seqeval')

args = TrainingArguments(output_dir='test_ner', num_train_epochs=3, 
                         learning_rate=2e-5, weight_decay=0.01,
                         per_device_train_batch_size=64, per_device_eval_batch_size=64,
                         evaluation_strategy='epoch', report_to='none')

trainer = Trainer(model=model, args=args, 
                  train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['valid'], 
                  data_collator=data_collator, tokenizer=tokenizer, 
                  compute_metrics=partial(compute_metrics, metric=metric, label_list=classlabel.names))

Training data size: 15 positives + 153 negatives: 100%|██████████| 5/5 [00:00<00:00, 131.50it/s]

Train.  Positive count: 146.  Negative count: 1229.
Valid.  Positive count: 15.  Negative count: 153.


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e7b03fe9b7f2a40f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,No log,0.067819,0.000000,0.000000,0.000000,0.987347,0.601700,279.199000
2,No log,0.046922,0.000000,0.000000,0.000000,0.987347,0.618600,271.596000
3,No log,0.040076,0.666667,0.214286,0.324324,0.988515,0.582000,288.676000


/opt/conda/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=66, training_loss=0.10843748034852924, metrics={'train_runtime': 44.9014, 'train_samples_per_second': 1.47, 'total_flos': 216059066545770.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1478688768, 'init_mem_gpu_alloc_delta': 431416832, 'init_mem_cpu_peaked_delta': 314843136, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 151523328, 'train_mem_gpu_alloc_delta': 1336198144, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 5876412416})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.04007585719227791,
 'eval_precision': 0.6666666666666666,
 'eval_recall': 0.21428571428571427,
 'eval_f1': 0.3243243243243243,
 'eval_accuracy': 0.9885146972941405,
 'eval_runtime': 0.5835,
 'eval_samples_per_second': 287.935,
 'epoch': 3.0,
 'eval_mem_cpu_alloc_delta': 49152,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_peaked_delta': 161348608}

In [ ]:
predictions, label_ids, _ = trainer.predict(tokenized_datasets['valid'])
predictions = np.argmax(predictions, axis=2)

true_predictions = [[classlabel.names[p] for p, i in zip(prediction, label_id) if i != -100]
                    for prediction, label_id in zip(predictions, label_ids)]

true_labels = [[classlabel.names[i] for i in label_id if i != -100] for label_id in label_ids]

print(true_predictions[2])
print(true_labels[2])

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'B', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
{'_': {'precision': 0.6666666666666666, 'recall': 0.21428571428571427, 'f1': 0.3243243243243243, 'number': 28}, 'overall_precision': 0.6666666666666666, 'overall_recall': 0.21428571428571427, 'overall_f1': 0.3243243243243243, 'overall_accuracy': 0.9885146972941405}


# Literal matching

In [ ]:
#export
def create_knowledge_bank(pth):
    '''
    Args:
        pth (str): Path to meta data like 'train.csv', which
        needs to have columns: 'dataset_title', 'dataset_label', and 'cleaned_label'.
        
    Returns:
        all_labels (set): All possible strings associated with a dataset from the meta data.
    '''
    df = load_train_meta(pth, group_id=False)
    all_labels = set()
    for label_1, label_2, label_3 in df[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
        all_labels.add(str(label_1).lower())
        all_labels.add(str(label_2).lower())
        all_labels.add(str(label_3).lower())
    return all_labels

In [ ]:
pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
all_labels = create_knowledge_bank(pth)
print(len(all_labels))
print(sorted(all_labels)[:10])

180
['2019 ncov complete genome sequences', '2019 ncov genome sequence', '2019 ncov genome sequences', '2019-ncov complete genome sequences', '2019-ncov genome sequence', '2019-ncov genome sequences', 'adni', 'advanced national seismic system (anss) comprehensive catalog (comcat)', 'advanced national seismic system anss comprehensive catalog comcat ', 'advanced national seismic system comprehensive catalog']


In [ ]:
#export
def literal_match(paper, all_labels):
    '''
    Args:
        paper ()
    '''
    text_1 = '. '.join(section['text'] for section in paper).lower()
    text_2 = clean_training_text(text_1, lower=True, total_clean=True)
    
    labels = set()
    for label in all_labels:
        if label in text_1 or label in text_2:
            labels.add(clean_training_text(label, lower=True, total_clean=True))
    return labels

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/test/', sample_submission.Id)

pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
all_labels = create_knowledge_bank(pth)

literal_preds = []
for paper_id in sample_submission.Id:
    paper = papers[paper_id]
    literal_preds.append('|'.join(literal_match(paper, all_labels)))
    
literal_preds

['adni|alzheimer s disease neuroimaging initiative adni',
 'common core of data|trends in international mathematics and science study|nces common core of data',
 'slosh model|noaa storm surge inundation|sea lake and overland surges from hurricanes',
 'rural urban continuum codes']

# Bert model prediction

In [ ]:
#export
def get_ner_inference_data(papers, sample_submission):
    '''
    Args:
        papers (dict): Each list in this dictionary consists of the section of a paper.
        sample_submission (pd.DataFrame): Competition 'sample_submission.csv'.
    Returns:
        test_rows (list): Each dict in this list is of the form: 
            {'tokens': ['goat', 'win', ...], 'tags': ['O', 'O', ...]}
            and represents a sentence.  
        paper_length (list): Number of sentences in each paper.
    '''
    test_rows = [] # test data in NER format
    paper_length = [] # store the number of sentences each paper has

    for paper_id in sample_submission['Id']:
        # load paper
        paper = papers[paper_id]

        # extract sentences
        sentences = [clean_training_text(sentence) for section in paper 
                     for sentence in section['text'].split('.')
                    ]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
        sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]

        # collect all sentences in json
        for sentence in sentences:
            sentence_words = sentence.split()
            dummy_tags = ['O']*len(sentence_words)
            test_rows.append({'tokens' : sentence_words, 'tags' : dummy_tags})

        # track which sentence belongs to which data point
        paper_length.append(len(sentences))

    print(f'total number of sentences: {len(test_rows)}')
    return test_rows, paper_length

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/test', sample_submission.Id)
test_rows, paper_length = get_ner_inference_data(papers, sample_submission)
print(test_rows[:3])
print(paper_length)

total number of sentences: 367
[{'tokens': ['A', 'recent', 'large', 'genomewide', 'association', 'study', 'GWAS', 'reported', 'a', 'genome', 'wide', 'significant', 'locus', 'for', 'years', 'of', 'education', 'which', 'subsequently', 'demonstrated', 'association', 'to', 'general', 'cognitive', 'ability', 'g', 'in', 'overlapping', 'cohorts'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'tokens': ['The', 'current', 'study', 'was', 'designed', 'to', 'test', 'whether', 'GWAS', 'hits', 'for', 'educational', 'attainment', 'are', 'involved', 'in', 'general', 'cognitive', 'ability', 'in', 'an', 'independent', 'large', 'scale', 'collection', 'of', 'cohorts'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, {'tokens': ['We', 'next', 'conducted', 'meta', 'analyses', 'with', '24', '189', 'in

In [ ]:

def kaggle_run_ner_predict(model_name_or_path='/kaggle/input/coleridge-bert-models/output', 
                           train_file='/kaggle/input/coleridge-bert-models/train_ner.json', 
                           validation_file='/kaggle/input/coleridge-bert-models/train_ner.json', 
                           test_file='./input_data/test_ner_input.json', 
                           output_dir='./pred'):
    '''
    Args:
        test_file (Path, str): Path to json file in which each row represents an input
            sample to the model (representing a sentence in this context).  Each row
            is a dictionary of the form:
            {'tokens': ['hi', 'there', ...], 'tags': ['O', 'O', ...]}
        output_dir (Path, str): Path to the directory in which prediction results are saved.
    '''
    os.environ["MODEL_PATH"] = f"{model_name_or_path}"
    os.environ["TRAIN_FILE"] = f"{train_file}"
    os.environ["VALIDATION_FILE"] = f"{validation_file}"
    os.environ["TEST_FILE"] = f"{test_file}"
    os.environ["OUTPUT_DIR"] = f"{output_dir}"
    
    ! python /kaggle/input/kaggle-ner-utils/kaggle_run_ner.py \
    --model_name_or_path "$MODEL_PATH" \
    --validation_file "$VALIDATION_FILE" \
    --train_file "$TRAIN_FILE" \
    --test_file "$TEST_FILE" \
    --output_dir "$OUTPUT_DIR" \
    --report_to 'none' \
    --seed 123 \
    --do_predict

def run_inference(test_rows, predict_batch=64_000, 
                  model_name_or_path='/kaggle/input/coleridge-bert-models/output', 
                  train_file='/kaggle/input/coleridge-bert-models/train_ner.json', 
                  validation_file='/kaggle/input/coleridge-bert-models/train_ner.json', 
                  test_file='./input_data/test_ner_input.json', 
                  output_dir='./pred'):
    '''
    '''
    test_file = Path(test_file)
    test_file.parent.mkdir(exist_ok=True, parents=True)
    
    bert_outputs = []
    for batch_begin in range(0, len(test_rows), predict_batch):
        # write data rows to input file
        with open(test_file, 'w') as f:
            for row in test_rows[batch_begin:batch_begin + predict_batch]:
                json.dump(row, f)
                f.write('\n')

        # remove output dir
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)

        # do predict
        kaggle_run_ner_predict(
            model_name_or_path=model_name_or_path, 
            train_file=train_file, validation_file=validation_file, test_file=test_file, 
            output_dir=output_dir)

        # read predictions
        with open(f'{output_dir}/test_predictions.txt') as f:
            this_preds = f.read().split('\n')[:-1]
            bert_outputs += [pred.split() for pred in this_preds]
    return bert_outputs

In [ ]:
predict_batch = 64_000 

model_name_or_path = '/kaggle/working/output/' #'/kaggle/input/coleridge-bert-models/output'
test_file = './input_data/test_ner_input.json'
train_file = 'train_ner.json' #'/kaggle/input/coleridge-bert-models/train_ner.json'
validation_file = 'train_ner.json' #'/kaggle/input/coleridge-bert-models/train_ner.json'
output_dir = './pred'

bert_outputs = run_inference(test_rows, predict_batch=predict_batch, 
                             model_name_or_path=model_name_or_path, 
                             test_file=test_file, train_file=train_file, validation_file=validation_file,
                             output_dir=output_dir)

2021-05-21 09:02:27.818742: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ca09f73f453ed05f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
Traceback (most recent call last):
  File "/kaggle/input/kaggle-ner-utils/kaggle_run_ner.py", line 501, in <module>
    main()
  File "/kaggle/input/kaggle-ner-utils/kaggle_run_ner.py", line 242, in main
    datasets = load_dataset(extension, data_files=data_files)
  File "/opt/conda/lib/python3.7/site-packages/datasets/load.py", line 751, in load_dataset
    ds = builder_instance.as_dataset(split=split, ignore_verifications=ignore_verifications, in_memory=keep_in_memory)
  File "/opt/conda/lib/python3.7/site-packages/datasets/builder.py", line 746, in as_dataset
    map_tuple=True,
  File "/opt/conda/lib/python3.7/site-packages/da

FileNotFoundError: [Errno 2] No such file or directory: './pred/test_predictions.txt'

In [ ]:
! ls {output_dir}/test_predictions.txt

ls: cannot access './pred/test_predictions.txt': No such file or directory


In [ ]:
#export
def get_bert_dataset_labels(test_rows, paper_length, bert_outputs):
    '''
    Returns:
        bert_dataset_labels (list): Each element is a set consisting of labels predicted
            by the model.
    '''
    test_sentences = [row['tokens'] for row in test_rows]
    
    bert_dataset_labels = [] # store all dataset labels for each publication

    for length in paper_length:
        labels = set()
        for sentence, pred in zip(test_sentences[:length], bert_outputs[:length]):
            curr_phrase = ''
            for word, tag in zip(sentence, pred):
                if tag == 'B': # start a new phrase
                    if curr_phrase:
                        labels.add(curr_phrase)
                        curr_phrase = ''
                    curr_phrase = word
                elif tag == 'I' and curr_phrase: # continue the phrase
                    curr_phrase += ' ' + word
                else: # end last phrase (if any)
                    if curr_phrase:
                        labels.add(curr_phrase)
                        curr_phrase = ''
            # check if the label is the suffix of the sentence
            if curr_phrase:
                labels.add(curr_phrase)
                curr_phrase = ''

        # record dataset labels for this publication
        bert_dataset_labels.append(labels)

        del test_sentences[:length], bert_outputs[:length]
        
    return bert_dataset_labels

In [ ]:
sentences = ['They do not present all the features', 
             'Despite the pretraining on the Tigers EcoNAX dataset',
             'Weirdly there has been lots of studies based on WGS Equality Definitiveness Dataset']
paper_length = [2, 1]
test_rows = [{'tokens': sentence.split(), 'tags': len(sentence.split()) * ['O']} 
             for sentence in sentences]
bert_outputs = [['O', 'O', 'O', 'B', 'I', 'I', 'O'],
                ['O', 'O', 'O', 'O', 'O', 'B', 'I', 'I'],
                ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I']]

for i, row in enumerate(test_rows):
    assert len(row['tokens']) == len(row['tags']) == len(bert_outputs[i])

bert_dataset_labels = get_bert_dataset_labels(test_rows, paper_length, bert_outputs)

In [ ]:
bert_dataset_labels

[{'Tigers EcoNAX dataset', 'present all the'},
 {'WGS Equality Definitiveness Dataset'}]

In [ ]:
#export
def jaccard_similarity(s1, s2):
    l1 = set(s1.split(" "))
    l2 = set(s2.split(" "))
    intersection = len(list(l1.intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

In [ ]:
jaccard_similarity('USGS Frog Counts Data', 'USGA Croc Counts Data') == 1 / 3

True

In [ ]:
#export
def filter_bert_labels(bert_dataset_labels):
    '''
    When several labels for a paper are too similar, keep just one of them.
    '''
    filtered_bert_labels = []

    for labels in bert_dataset_labels:
        filtered = []

        for label in sorted(labels, key=len):
            label = clean_training_text(label, lower=True)
            if len(filtered) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered):
                filtered.append(label)

        filtered_bert_labels.append('|'.join(filtered))
    return filtered_bert_labels

In [ ]:
bert_dataset_labels = [{'moma artists catalogue', 'moma artists', 'housing market'},
                       {'rhs flowers fertiliser index', 'deep sea rock salts', 'rhs fertiliser index'}]

filter_bert_labels(bert_dataset_labels)

['moma artists|housing market|moma artists catalogue',
 'deep sea rock salts|rhs fertiliser index']

# Overall prediction for submission

In [ ]:
#export
def combine_matching_and_bert(literal_preds, filtererd_bert_labels):
    final_predictions = []
    for literal_match, bert_pred in zip(literal_preds, filtered_bert_labels):
        if literal_match:
            final_predictions.append(literal_match)
        else:
            final_predictions.append(bert_pred)
    return final_predictions

In [ ]:
literal_preds = ['mongolian racing cars|reallife headphones', '']
filtered_bert_labels = ['data|dataset', 'hifi dataset|headphones collection data']
combine_matching_and_bert(literal_preds, filtered_bert_labels)

['mongolian racing cars|reallife headphones',
 'hifi dataset|headphones collection data']

# Reference
- https://www.kaggle.com/tungmphung/pytorch-bert-for-named-entity-recognition/notebook
- https://www.kaggle.com/tungmphung/coleridge-matching-bert-ner/notebook
- https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb